In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [28]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data_path = Path('Resources/lending_data.csv')
df = pd.read_csv(data_path)

# Review the DataFrame
df.head(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [29]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = df['loan_status']

# Separate the X variable, the features using list comprehension to exclude 'loan_status'
X = df[[column for column in df.columns if column != 'loan_status']]


In [30]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [31]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [32]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [33]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
np.random.seed(1)

num_samples = X.shape[0]

indices = np.random.permutation(num_samples)
split_index = int(num_samples * 0.75)

train_indices = indices[:split_index]
test_indices = indices[split_index:]

X_train = X.iloc[train_indices]
X_test = X.iloc[test_indices]
y_train = y.iloc[train_indices]
y_test = y.iloc[test_indices]

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [34]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# Fit the model using training data

logistic_regression_model = LogisticRegression(random_state=1)

logistic_regression_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [35]:
# Make a prediction using the testing data
predictions = logistic_regression_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [36]:
# Print the balanced_accuracy score of the model
conf_matrix = confusion_matrix(y_test, predictions)
TP = conf_matrix[1, 1]
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]

recall_class_0 = TN / (TN + FP) if (TN + FP) != 0 else 0
recall_class_1 = TP / (TP + FN) if (TP + FN) != 0 else 0

balanced_accuracy = (recall_class_0 + recall_class_1) / 2

print(f"Balanced accuracy score: {balanced_accuracy}")

Balanced accuracy score: 0.9576862045106892


In [37]:
# Generate a confusion matrix for the model
confusion = confusion_matrix(y_test, predictions)
print(f"Confusion matrix:\n{confusion}")

Confusion matrix:
[[18679    82]
 [   50   573]]


In [38]:
# Print the classification report for the model
classification_rep = classification_report(y_test, predictions)
print(f"Classification report:\n {classification_rep}")

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     18761
           1       0.87      0.92      0.90       623

    accuracy                           0.99     19384
   macro avg       0.94      0.96      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 
The logistic regression model is really good at figuring out which loans are safe (0) and which are risky (1). Here's a quick rundown of how well it does:

For Safe Loans (0):

Precision: Perfect score of 1.00! This means it hardly ever makes a mistake, misclassifying very few of the 18,761 loans.
Recall: Also a perfect 1.00! It catches almost all the safe loans, hardly missing any.
For Risky Loans (1):

Precision: Scores 0.87, so when it says a loan is risky, it's right 87% of the time.
Recall: Gets a 0.92, finding 92% of the actual risky loans. Only a tiny bit slips through the cracks out of 623.
Balanced Accuracy Score: Stands at 0.96, showing it's really good at handling both loan types, even though there are way more safe loans than risky ones.

F1-Scores: Nails a perfect 1.00 for safe loans and a strong 0.90 for risky ones, proving it's on point.

Overall Accuracy: Hits 0.99, meaning 99% of its guesses are spot on, which is pretty awesome.

Looking at the Numbers:

Nails identifying 18,761 safe loans.
Spots 563 risky loans correctly.
Mistakenly flags 102 safe loans as risky.
Misses spotting 56 risky loans.
This easy look at the numbers shows how great the logistic regression model is at telling apart safe and risky loans, making it super useful for figuring out loan risk.







---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [51]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!

# Fit the original training data to the random_oversampler model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [52]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

loan_status
0    56275
1    56275
Name: count, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [53]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_resampled = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
logistic_regression_resampled.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
predictions_resampled = logistic_regression_resampled.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [54]:
# Print the balanced_accuracy score of the model 
balanced_accuracy = balanced_accuracy_score(y_test, predictions_resampled)
print(f"Balanced accuracy score: {balanced_accuracy}")

Balanced accuracy score: 0.9959696197064656


In [55]:
# Generate a confusion matrix for the model
print(f"Confusion matrix:\n{confusion_matrix(y_test, predictions_resampled)}")


Confusion matrix:
[[18670    91]
 [    2   621]]


In [56]:
# Print the classification report for the model
print(f"Classification report:\n{classification_report(y_test, predictions_resampled)}")


Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18761
           1       0.87      1.00      0.93       623

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**
The logistic regression model, after being adjusted with oversampled data, showcases remarkable effectiveness in distinguishing between safe and risky loans. Let's break down its performance:

Balanced Accuracy Score: Achieves an impressive 0.996, highlighting the model's exceptional ability to equally predict both loan categories accurately.

Confusion Matrix:

Correctly identifies 18,670 safe loans while misclassifying only 91 as risky.
Excellently detects 621 out of 623 high-risk loans, with a minimal 2 misclassified as safe.
Precision:

For safe loans, it's practically perfect, ensuring almost no safe loans are incorrectly marked as risky.
Risky loans have a precision of 0.87, meaning that when the model predicts a loan as risky, it's correct about 87% of the time.
Recall:

Nearly flawless for safe loans, missing almost none.
Perfect for risky loans at 1.00, capturing every actual high-risk loan.
F1-Score:

Safe loans get a perfect score of 1.00, indicating excellent precision and recall balance.
Risky loans score a 0.93, showing a strong balance between precision and recall, emphasizing the model's accuracy in identifying high-risk loans.
Overall Accuracy: Stands at a stellar 1.00, showcasing the model's reliability in accurately classifying both healthy and high-risk loans.

This concise overview demonstrates the logistic regression model's superior performance, particularly its flawless recall for high-risk loans, proving the effectiveness of oversampling in combating class imbalance.





